![This is an image](Quant-Trading.jpg)

<font size="3">
Please visit our website <a href="https://www.quant-trading.co" target="_blank">quant-trading.co</a> for more tools on quantitative finance and data science.
</font>

# **HOW TO DOWNLOAD DATA FROM THE FDIC API**

## **¿What can you download using this API?**

<font size="3"> The Federal Deposit Insurance Corporation (FDIC) is an independent agency created by Congress to maintain stability and public confidence in the nation's financial system. To accomplish this mission, the FDIC insures deposits; examines and supervises financial institutions for safety, soundness, and consumer protection; makes large and complex financial institutions resolvable; and manages receiverships. FDIC’s application programming interface (API) lets developers access FDIC’s publically available bank data. In this notebook we will explain how to download data from the FDIC API. Everything you need to integrate with this API is available below on their  <a href="https://banks.data.fdic.gov/docs/" target="_blank">website</a>.<br><br> 
        
<font size="3"> The API is open to the public and does not currently require any authentication or API keys to access. You can get information on the following topics<br><br>


    - Institutions (CSV format) - Listing of FDIC-insured banking institution demographic and headquarter location data.
    - Locations (CSV format) - Listing of FDIC-insured banks branches and locations.
    - History Events Definitions (CSV format) - Definitions of fields in History data.
    - Demographics Data (YAML format) - for details on demographics.
    - Etc
<br><br>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import requests
import io
import matplotlib.pyplot as plt

import matplotlib.font_manager as font_manager
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter

## **Find an specific bank**

<font size="3">You can look in the institutions file for a particular bank. For example, we show here how to look por Chase Bank. We will get the following URL<br><br>

In [9]:
name = 'Chase'

url = "https://banks.data.fdic.gov/api/institutions?"
url += f"search=NAME%3A%20%27{name.replace(' ', '%20')}%27"

url

'https://banks.data.fdic.gov/api/institutions?search=NAME%3A%20%27Chase%27'

## **Find specific fields and limit the number of rows**

<font size="3">We can also look for some particular information of the bank. In this case we are looking for the Name, City and State<br><br>

In [11]:
fields=["NAME", "CITY", "STATE"]
limit=100

url += f"&fields=FED_RSSD%2CREPDTE%2C{('%2C'.join(fields))}&limit={limit}&format=csv&download=false&filename=data_file"

response = requests.get(url)
df = pd.read_csv(io.StringIO(response.text))
df

,CITY,FED_RSSD,ID,NAME,REPDTE
0,Columbus,852218,628,"JPMorgan Chase Bank, National Association",12/31/2023
1,Wilmington,489913,23702,"Chase Bank USA, National Association",03/31/2019
2,Austin,63256,20834,"Frost Bank North Austin, National Association",12/31/1988
3,Baltimore,513779,26413,The Chase Manhattan Bank of Maryland,09/30/1995
4,Mclean,686477,32324,"Chevy Chase Bank, F.S.B.",06/30/2009
5,Chase,739551,16697,Bank Chase,06/30/1996
6,Miami,754778,29001,"Chase Federal Bank, a Federal Savings Bank",06/30/1996
7,Hatboro,30874,28888,Fox Chase Bank,06/30/2016
8,Palm Beach,146739,24381,"The Chase Manhattan Trust Company of Florida, ...",12/31/1987
9,Houston,43557,3263,"Chase Bank of Texas, National Association",06/30/2000


## **Create a function to download generic data**

<font size="3">We can create a function to download the data por a particular bank. It is similar to what we have done above. Let's see an example below:<br><br>

In [2]:
def getInstitution(name=None, IDRSSD_or_CERT=None, fields=None, IDRSSD=True, limit=10000):
    assert fields is not None, "Fields cannot be empty"
    if name is not None and IDRSSD_or_CERT is not None:
        raise ValueError("Please use only 'name' or 'IDRRSD_or_CERT' parameter.")
    
    url = "https://banks.data.fdic.gov/api/institutions?"
    if name is not None:
        url += f"search=NAME%3A%20%27{name.replace(' ', '%20')}%27"
    elif IDRSSD_or_CERT is not None:
        if IDRSSD:
            url += f"filters=FED_RSSD%3A%20{IDRSSD_or_CERT}"
        else:
            url += f"filters=CERT%3A%20{IDRSSD_or_CERT}"
    else:
        raise ValueError("Please provide either 'name' or 'IDRSSD_or_CERT' parameter.")
    
    url += f"&fields=FED_RSSD%2CREPDTE%2C{('%2C'.join(fields))}&limit={limit}&format=csv&download=false&filename=data_file"
    
    try:
        response = requests.get(url)
        df = pd.read_csv(io.StringIO(response.text))
        df = df.drop(["ID"], axis=1)
        df = df.rename(columns={"FED_RSSD": "IDRSSD", "REPDTE": "DATE"})
        df["DATE"] = pd.to_datetime(df["DATE"], format="%m/%d/%Y")
        return df
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return None

print(getInstitution(name = 'Chase', IDRSSD_or_CERT=None, fields=["NAME", "CITY", "STATE"], limit=100))

           CITY   IDRSSD                                               NAME  \
0      Columbus   852218          JPMorgan Chase Bank, National Association   
1    Wilmington   489913               Chase Bank USA, National Association   
2        Austin    63256      Frost Bank North Austin, National Association   
3     Baltimore   513779               The Chase Manhattan Bank of Maryland   
4        Mclean   686477                           Chevy Chase Bank, F.S.B.   
5         Chase   739551                                         Bank Chase   
6         Miami   754778         Chase Federal Bank, a Federal Savings Bank   
7       Hatboro    30874                                     Fox Chase Bank   
8    Palm Beach   146739  The Chase Manhattan Trust Company of Florida, ...   
9       Houston    43557          Chase Bank of Texas, National Association   
10        Tampa   617734  Chase Manhattan Private Bank, National Associa...   
11  Los Angeles  2607382   J. P. Morgan Trust Compan

## **Get specific financial information**

<font size="3">Now we can get particular financial information from a bank. In this case we are going to use the IDRSSD number 852218, which corresponds to JP Morgan Chase Bank. We will download the following fields: ["ASSET", "DEP","CERT","ROE","NAME","EQTOT"]. If you want to know which fields you can search, please look at this <a href="https://banks.data.fdic.gov/docs/#/Financials/getFinancials" target="_blank">link</a>  <br><br>

In [3]:
ID_code = 852218


def getFinancials(IDRSSD_or_CERT, metrics, limit=1, IDRSSD=True, date_range=None):
    assert IDRSSD_or_CERT is not None and metrics is not None, "IDRSSD_or_CERT and metrics cannot be empty"
    assert isinstance(IDRSSD_or_CERT, int), "IDRSSD_or_CERT must be a numeric value"
    if date_range is not None:
        assert len(date_range) == 2, "Date range must have two values"
    
    url = f"https://banks.data.fdic.gov/api/financials?filters={('RSSDID' if IDRSSD else 'CERT')}%3A%20{IDRSSD_or_CERT}"
    if date_range is not None:
        url += f"%20AND%20REPDTE%3A%5B{date_range[0]}%20TO%20{date_range[1]}%5D"
    url += f"&fields=RSSDID%2CREPDTE%2C{('%2C'.join(metrics))}&sort_by=REPDTE&sort_order=DESC&limit={limit}&offset=0&agg_term_fields=REPDTE&format=csv&download=false&filename=data_file"
    
    try:
        response = requests.get(url)
        df = pd.read_csv(io.StringIO(response.text))
        df = df.assign(DATE=pd.to_datetime(df["REPDTE"], format="%Y%m%d")).drop(["REPDTE", "ID"], axis=1).rename(columns={"RSSDID": "IDRSSD"})
        return df
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return None

print(getFinancials(IDRSSD_or_CERT = ID_code, metrics=["ASSET", "DEP","CERT","ROE","NAME","EQTOT"], limit=1000, date_range=["2000-01-01", "*"]))

         ASSET  CERT         DEP      EQTOT                    NAME    ROE  \
0   3395126000   628  2498231000  299323000  JPMORGAN CHASE BANK NA  15.37   
1   3385581000   628  2459921000  317842000  JPMORGAN CHASE BANK NA  16.65   
2   3382195000   628  2505650000  315407000  JPMORGAN CHASE BANK NA  16.62   
3   3267963000   628  2472913000  309052000  JPMORGAN CHASE BANK NA  15.30   
4   3201942000   628  2440722000  303672000  JPMORGAN CHASE BANK NA  11.45   
..         ...   ...         ...        ...                     ...    ...   
91   400623000   628   243608000   23087000    CHASE MANHATTAN BANK  22.05   
92   377116000   628   238835000   22201000    CHASE MANHATTAN BANK  17.09   
93   346204000   628   212749000   21454000    CHASE MANHATTAN BANK  16.81   
94   320476000   628   195919000   19283000    CHASE MANHATTAN BANK  19.97   
95   312205000   628   184088000   18767000    CHASE MANHATTAN BANK  19.83   

    IDRSSD       DATE  
0   852218 2023-12-31  
1   852218 2023

## **Put the info in a DataFrame**

<font size="3">Just make it more concise this way:<br><br>

In [4]:
df_selected_bank = getFinancials(IDRSSD_or_CERT = ID_code, metrics=["ASSET", "DEP","CERT","ROE","NAME","EQTOT"], limit=1000, date_range=["2000-01-01", "*"])
df_selected_bank

,ASSET,CERT,DEP,EQTOT,NAME,ROE,IDRSSD,DATE
0,3395126000,628,2498231000,299323000,JPMORGAN CHASE BANK NA,15.37,852218,2023-12-31
1,3385581000,628,2459921000,317842000,JPMORGAN CHASE BANK NA,16.65,852218,2023-09-30
2,3382195000,628,2505650000,315407000,JPMORGAN CHASE BANK NA,16.62,852218,2023-06-30
3,3267963000,628,2472913000,309052000,JPMORGAN CHASE BANK NA,15.30,852218,2023-03-31
4,3201942000,628,2440722000,303672000,JPMORGAN CHASE BANK NA,11.45,852218,2022-12-31
...,...,...,...,...,...,...,...,...
91,400623000,628,243608000,23087000,CHASE MANHATTAN BANK,22.05,852218,2001-03-31
92,377116000,628,238835000,22201000,CHASE MANHATTAN BANK,17.09,852218,2000-12-31
93,346204000,628,212749000,21454000,CHASE MANHATTAN BANK,16.81,852218,2000-09-30
94,320476000,628,195919000,19283000,CHASE MANHATTAN BANK,19.97,852218,2000-06-30


If this content is helpful and you want to make a donation please click on the button

[![paypal](https://www.paypalobjects.com/en_US/i/btn/btn_donateCC_LG.gif)](https://www.paypal.com/cgi-bin/webscr?cmd=_s-xclick&hosted_button_id=29CVY97MEQ9BY)